# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier as a reward function.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets the start of a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from utils import set_random_seed
set_random_seed(20200819)

### Import dependencies

In [2]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

/home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


/home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/yadong/software/miniconda3/envs/lightning did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/home/yadong/.oh-my-zsh/functions'), PosixPath('/usr/share/zsh/vendor-functions'), PosixPath('/home/yadong/.oh-my-zsh/completions')}
  warn(msg)
/home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/yadong/software/miniconda3/envs/lightning/

### Configuration

In [3]:
config = PPOConfig(
    # model_name="/home/yadong/workspace/llama/lightning_train/weights/vqg/base_val_loss=0.641_epoch=0.ckpt.full",
    learning_rate=1.41e-5,
    # log_with="wandb",
)


In [4]:
import wandb

wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1397280422. Use `wandb login --relogin` to force relogin


### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [5]:
model_name = "/home/yadong/workspace/llama/lightning_train/weights/vqg/base_val_loss=0.641_epoch=0.ckpt.full"

model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("/vault/pretrains/huggingface/gpt2")

tokenizer.pad_token = tokenizer.eos_token

### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [6]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer)

/home/yadong/software/miniconda3/envs/lightning/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:238: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [7]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [8]:
from datas import load_datamodule
from omegaconf import OmegaConf
data_config = OmegaConf.load("config/data/vqg_vqa-score.yml")
data_module = load_datamodule(data_config, tokenizer, 256)
data_module.setup("fit")
dataloader = data_module.train_dataloader()

train_length: 8108
valid_length: 901


In [9]:
from models.vqg import VQAConfidenceRewardModule
reward_model = VQAConfidenceRewardModule("http://localhost:9401", 16)

In [10]:
generation_kwargs = {
    "max_length": 256,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": False,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in enumerate(tqdm(dataloader)):
    # query_tensors = batch["input_ids"]
    batch["query"] = batch["inputs"]
    query_tensors = [tokenizer(i, return_tensors="pt")["input_ids"].squeeze(dim=0) for i in batch["query"]]

    #### Get response from gpt2
    response_tensors = []
    # for query in query_tensors:
    #     gen_len = output_length_sampler()
    #     generation_kwargs["max_new_tokens"] = gen_len
    #     response = ppo_trainer.generate(query, **generation_kwargs)
    #     response_tensors.append(response.squeeze()[-gen_len:])
    response = ppo_trainer.generate(query_tensors, **generation_kwargs)
    response_tensors = [o[len(i):] for i,o in zip(query_tensors, response)]
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    output = zip(batch["query"], batch["response"], batch["origins"])
    rewards = reward_model(output)
    # pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    # rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    rewards = torch.tensor(rewards)
    stats["env/reward_mean"] = torch.mean(rewards).cpu().numpy().item()
    stats["env/reward_std"] = torch.std(rewards).cpu().numpy().item()
    stats["env/reward_dist"] = rewards.cpu().numpy()
    wandb.log(stats)
    # ppo_trainer.log_stats(stats, batch, rewards)

  9%|▉         | 3/32 [03:24<32:49, 67.91s/it]